In [1]:
import sys
import glob
sys.path.append("../")

path_dir_test_html = "../test_data/html"
list_fpath_test_html = glob.glob(path_dir_test_html + "/*.html")

print(list_fpath_test_html)

['../test_data/html\\20250326_174714_435197_3cf46808_dbeb97ba.html', '../test_data/html\\20250326_174714_596617_803c4ec8_09171200.html', '../test_data/html\\20250326_174714_770624_135bffe2_f36a1545.html', '../test_data/html\\20250326_174714_851857_c459903a_7821225b.html', '../test_data/html\\20250326_174714_933859_364b10b1_71653482.html', '../test_data/html\\20250326_174715_139110_dcda1a47_1ad4404b.html', '../test_data/html\\20250326_174715_202036_2c04c577_2b955057.html', '../test_data/html\\20250326_174715_284857_26d30523_98ffa6a3.html', '../test_data/html\\20250326_174715_496479_8e7d7493_d5087d63.html', '../test_data/html\\20250326_174715_594481_7ee9bcbd_ddea7b26.html']


In [2]:
from docmesh.extractor.HTMLFileInspector import HTMLFileInspector
from docmesh.text_split import HTMLContentLoader
from docmesh.text_split.DocumentMerger import DocumentMerger
from docmesh.text_split.DocumentChunkSplitter import DocumentChunkSplitter
from docmesh.embedding import EmbeddingModelFactory
from docmesh.vector_store import VectorStoreFactory

In [3]:
# 파일 로드 객체 생성
html_file_loader = HTMLFileInspector()
html_content_loader = HTMLContentLoader()

# 전처리 객체 생성
doc_merger = DocumentMerger(min_tokens=500)
doc_splitter = DocumentChunkSplitter(chunk_size=1000, chunk_overlap=100)
# EmbeddingModelFactory를 통해 임베딩 모델 생성
embedding_model = EmbeddingModelFactory.create_embedding_model(
    provider="openai", model_name="text-embedding-ada-002"
)
# VectorStoreFactory를 통해 벡터 스토어 생성
vector_store = VectorStoreFactory.create_vector_store(
    provider="faiss", embedding_model=embedding_model, path="langchain_doc.faiss", 
)

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [4]:
for fpath in list_fpath_test_html:
    url, html = html_file_loader.read_html(fpath)
    docs_langchain = html_content_loader.load(url, html)
    
    docs_merged = doc_merger.merge_documents(docs_langchain) # 최소 토큰 기준으로 chunk 병합
    docs_splitted = doc_splitter.chunkify(docs_merged) # 최대 토큰 기준으로 chunk 분리 및 overlap 생성
    
    vector_store.add_documents(docs_splitted)

In [5]:
vector_store.save()